In [39]:
import pandas as pd
import numpy as np
import datetime

In [65]:
df = pd.read_excel("XLS_takehome.xlsx")

In [66]:
df

,dateordered,datereturned,orderstatus,orders
0,2016-12-31,2017-01-24,returned,1
1,2016-12-31,NaT,complete,8
2,2016-12-30,NaT,complete,14
3,2016-12-29,NaT,complete,14
4,2016-12-28,2016-12-29,returned,1
5,2016-12-28,NaT,complete,23
6,2016-12-27,2017-01-28,returned,1
7,2016-12-27,2017-01-25,returned,1
8,2016-12-27,2017-01-15,returned,1
9,2016-12-27,2017-01-05,returned,1


Since we want to measure the metrics by month, I extracted the `month_ordered` and `month_returned` as two separate columns. Meanwhile, I calculated the difference between `dateordered` and `datereturned`, for future analysis purpose.

In [70]:
# Add month column to aggregate by month
df['month_ordered'] = [d.date().month for d in df['dateordered']]
df['month_returned'] = [d.date().month for d in df['datereturned']]

# Difference between return and order data (if applicable)
df['difference'] = df['datereturned'] - df['dateordered'] 

In [71]:
df

,dateordered,datereturned,orderstatus,orders,month_ordered,month_returned,difference
0,2016-12-31,2017-01-24,returned,1,12,1.0,24 days
1,2016-12-31,NaT,complete,8,12,NaN,NaT
2,2016-12-30,NaT,complete,14,12,NaN,NaT
3,2016-12-29,NaT,complete,14,12,NaN,NaT
4,2016-12-28,2016-12-29,returned,1,12,12.0,1 days
5,2016-12-28,NaT,complete,23,12,NaN,NaT
6,2016-12-27,2017-01-28,returned,1,12,1.0,32 days
7,2016-12-27,2017-01-25,returned,1,12,1.0,29 days
8,2016-12-27,2017-01-15,returned,1,12,1.0,19 days
9,2016-12-27,2017-01-05,returned,1,12,1.0,9 days


One thing to notice is that sometimes the difference can be very large (more than 90 days), which means the customer returned the product after using it for quite a long time. In such cases, I would count that into the ordered month. For example, an order was placed in August, but returned in November, then this case will be counted towards **August**.

I first aggregrated orders (ordered/ returned) by month, and merge them together to calculate the return rate.

One thing to note is that

https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/

In [87]:
# Get the total orders per month
temp1 = df.groupby('month_ordered')['orders'].sum()

# Get the sum of the orders per month
temp2 = df.loc[df['orderstatus'] == "returned"].groupby('month_ordered')['orders'].sum()

# Merge two
print(pd.DataFrame({"Month": range(8, 13), "Total": temp1, "Returned": temp2}).reset_index(drop = True))

   Month  Returned  Total
0      8        10    149
1      9         9    143
2     10        10    180
3     11        12    226
4     12        19    395
